# Simulations with reinforcement learning

In this notebook you can run simulations with 5 different reinforcement learning algorithms. There are different example buildings that can be simulated.

<br>

Things to focus on:
-  Why is it making more sub_runs than episodes??
-  There are only 10 episodes sub-files that stay. The rest is deleted??
-  Better to make a list with the mean rewards, so we can make plots after running.
-  Now, the room is always on temperature, how to see this. Weekends off? only on on working hours? 





In [14]:
import sinergym
from sinergym.utils.callbacks import LoggerEvalCallback, LoggerCallback
from sinergym.utils.rewards import *
from sinergym.utils.wrappers import LoggerWrapper
from sinergym.utils.constants import RANGES_5ZONE
from datetime import datetime
import gym
from stable_baselines3 import DQN, DDPG, PPO, A2C, SAC, TD3 

from stable_baselines3.common.logger import configure

from stable_baselines3.common.callbacks import CallbackList
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
import mlflow

Next you can set different variables for het simulation. You can choose the period that you want to simulate.

(For now the year is always 1991, has something to do with the current weather file, which is standard in the environment. Can be changed later on)

<br>

You also set the reward function here. Now it is set to exponential.





In [4]:
#environment = "Eplus-demo-v1"
environment  = "Eplus-5Zone-hot-continuous-v1"
#environment = "Eplus-5Zone-mixed- continuous-v1"



episodes = 1
experiment_date = datetime.today().strftime('%Y-%m-%d %H:%M')

#choose the simulation period
begin_day = 1
begin_month = 1
begin_year = 2022
end_day = 1
end_month = 2
end_year = 2022

# register run name
name = F"{environment}-episodes_{episodes}({experiment_date})"


# Set to one month only to reduce running time
extra_params={'timesteps_per_hour' : 4,
              'runperiod' : (begin_day,begin_month,begin_year,end_day,end_month,end_year)}

with mlflow.start_run(run_name=name):
    env = gym.make(environment, reward=LinearReward)
    env = LoggerWrapper(env)

    # Defining model(algorithm)
    model = PPO('MlpPolicy', env)

    # Calculating n_timesteps_episode for training
    n_timesteps_episode = env.simulator._eplus_one_epi_len / \
                          env.simulator._eplus_run_stepsize
    timesteps = episodes * n_timesteps_episode

    # For callbacks processing
    env_vec = DummyVecEnv([lambda: env])

    # Using Callbacks for training
    callbacks = []

    # Set up Evaluation and saving best model
    eval_callback = LoggerEvalCallback(
        env_vec,
        best_model_save_path='best_model/' + name + '/',
        log_path='best_model/' + name + '/',
        eval_freq=n_timesteps_episode * 2,
        deterministic=True,
        render=False,
        n_eval_episodes=2)
    callbacks.append(eval_callback)

    callback = CallbackList(callbacks)

    # Training
    model.learn(
        total_timesteps=timesteps,
        callback=callback,
        log_interval=1)
    model.save(env.simulator._env_working_dir_parent + '/' + name)

    # End mlflow run
    mlflow.end_run()


[2023-01-11 10:19:38,315] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-01-11 10:19:38,315] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-01-11 10:19:38,326] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-01-11 10:19:38,326] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-01-11 10:19:38,343] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-01-11 10:19:38,343] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-01-11 10:19:38,357] EPLUS_ENV_5Zone-hot-continuous-v1_Ma

/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


[2023-01-11 10:19:38,973] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-01-11 10:19:38,973] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-01-11 10:19:39,209] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-continuous-v1-res56/Eplus-env-sub_run1
[2023-01-11 10:19:39,209] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-continuous-v1-res56/Eplus-env-sub_run1


KeyboardInterrupt: 

In [70]:
# schedulers=env.get_schedulers()
# print(schedulers)
# actuators=env.get_schedulers(path='./example.xlsx')

Tenserflow check


In [15]:
environment = "Eplus-demo-v1"
episodes = 4
experiment_date = datetime.today().strftime('%Y-%m-%d %H:%M')

# register run name
name = F"DQN-{environment}-episodes_{episodes}({experiment_date})"

env = gym.make(environment, reward=LinearReward)
env = LoggerWrapper(env)

[2023-01-11 10:30:38,092] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-01-11 10:30:38,092] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-01-11 10:30:38,092] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-01-11 10:30:38,101] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-01-11 10:30:38,101] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-01-11 10:30:38,101] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-01-11 10:30:38,123] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-01-11 10:30:38,123] E

In [16]:
tensorboard_path='./tensorboard_log'
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=tensorboard_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [17]:
n_timesteps_episode = env.simulator._eplus_one_epi_len / \
                      env.simulator._eplus_run_stepsize
timesteps = episodes * n_timesteps_episode
env_vec = DummyVecEnv([lambda: env])

In [18]:
callbacks = []

# Set up Evaluation and saving best model
log_callback = LoggerCallback(True)
callbacks.append(log_callback)
# lets change default dir for TensorboardFormatLogger only
tb_path = tensorboard_path + '/' + name
new_logger = configure(tb_path, ["tensorboard"])
model.set_logger(new_logger)

callback = CallbackList(callbacks)

2023-01-11 10:30:40.014335: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-11 10:30:48.982620: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-11 10:30:59.368498: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-11 10:30:59.369628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or direc

In [19]:
model.learn(
    total_timesteps=timesteps,
    callback=callback,
    log_interval=1)
model.save(env.simulator._env_working_dir_parent + '/' + name)
env.close()

[2023-01-11 10:31:06,741] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-01-11 10:31:06,741] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-01-11 10:31:06,741] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-01-11 10:31:06,957] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res7/Eplus-env-sub_run1
[2023-01-11 10:31:06,957] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res7/Eplus-env-sub_run1
[2023-01-11 10:31:06,957] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res7/Eplus-env-sub_run1
[2023-01-11 10:38:44,232] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-01-11 10:38:44,232] EPLUS_ENV_demo-v

KeyboardInterrupt: 